## Query Processing Module

#### Query Input

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/deepti-
[nltk_data]     saravanan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from nltk.corpus import stopwords
from nltk.metrics import edit_distance
#Definitions dictionary
import os, json
import pandas as pd
import pickle
defdict = {}
path_to_json = './Definitions/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
for i in json_files:
    with open('./Definitions/'+i) as json_file: 
        data = json.load(json_file)
    defdict.update(data)
definitions = list(defdict.keys())
defvalues = list(defdict.values())

docs = ["Issue and Listing of Non-Convertible Redeemable Preference Shares", "Investment Advisers", "Depositories and Participants", "Mutual Funds", "Employees' Service", "Substantial Acquisition of Shares and Takeovers", "Appointment of Administrator and Procedure for Refunding to the Investors", "Prohibition of Fraudulent and Unfair Trade Practices relating to Securities Market", "Know Your Client", "Prohibition of Insider Trading", "Merchant Bankers", "Issue and Listing of Securitised Debt Instruments and Security Receipts", "Delisting of Equity Shares", "Issue of Capital and Disclosure Requirements", "Foreign Venture Capital Investor", "Procedure for Board Meetings", "Custodian", "Ombudsman", "Investor Protection and Education Fund", "Foreign Portfolio Investors", "Issue of Sweat Equity", "Collective Investment Scheme", "Portfolio Managers", "Research Analysts", "Procedure for Search and Seizure", "Issue Of Capital And Disclosure Requirements", "Share Based Employee Benefits", "Debenture Trustees", "Alternative Investment Funds", "Stock Exchanges and Clearing Corporations", " Self Regulatory Organisations", "Settlement Proceedings", "Issue and Listing of Municipal Debt Securities", "Buy-back of Securities", "Issue and Listing of Debt Securities", "Infrastructure Investment Trusts", " Stock Brokers", "Listing Obligations and Disclosure Requirements", "Registrars to an Issue and Share Transfer Agents", "Real Estate Investment Trusts", "Intermediaries", "Certification of Associated Persons in the Securities Markets", "Credit Rating Agencies", "Regulatory Fee on Stock Exchanges", "Underwriters", "Buy Back Of Securities", "Bankers to an Issue", "Central Database of Market Participants"]

#Regulations
finalclean48 = []
with open('cleanedregulations48.pkl','rb') as f:
    finalclean48 = pickle.load(f)    
docregs = finalclean48

#Topics documentwise
finalcleantopics48 = []
with open('cleanedregtopics48.pkl','rb') as f:
    finalcleantopics48 = pickle.load(f)    
finaltopics = finalcleantopics48

#vocab definitions
mainvocab = []
with open('mainvocab.pkl','rb') as f:
    mainvocab = pickle.load(f) 

vocabdef = []
with open('vocabdef.pkl','rb') as f:
    vocabdef = pickle.load(f) 

#Additional Documents

with open('concept_filenames.pkl','rb') as f:
    cfile = pickle.load(f) 
with open('concept_sentences.pkl','rb') as f:
    csent = pickle.load(f) 
with open('concept_text.pkl','rb') as f:
    ctext = pickle.load(f) 

with open('informal_filenames.pkl','rb') as f:
    ifile = pickle.load(f) 
with open('informal_sentences.pkl','rb') as f:
    isent = pickle.load(f) 
with open('informal_text.pkl','rb') as f:
    itext = pickle.load(f) 
    
with open('miscvocab.pkl', 'rb') as f:
    miscvocab = pickle.load(f)
    
#Legal Case files
with open('case_filenames.pkl','rb') as f:
    lfile = pickle.load(f) 
with open('case_sentences.pkl','rb') as f:
    lsent = pickle.load(f) 
with open('case_text.pkl','rb') as f:
    ltext = pickle.load(f) 
    
with open('casevocab.pkl', 'rb') as f:
    casevocab = pickle.load(f)

import spacy
#nlp = spacy.load('en_core_web_lg')
nlp = spacy.load('en')

#Legal pre-trained model for synonyms
from gensim.models.keyedvectors import KeyedVectors
#m = KeyedVectors.load_word2vec_format('/home/deepti-saravanan/Music/raw-legal/no_replacements/legalmodeltext.txt',binary=False, unicode_errors='ignore')
#modelvocab = list(m.vocab.keys())

from nltk.corpus import wordnet
from nltk import ngrams
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import math
import numpy as np
from collections import Counter
from nltk.corpus import stopwords
from sklearn.feature_extraction import stop_words
STOPWORDS = set(
    list(stop_words.ENGLISH_STOP_WORDS) +\
    stopwords.words('english') +\
    ['mm', 'section', 'subsection', 'schedule', '-PRON-', 'chapter', 'regulation', 'repealed', 'thereto','unpublishe', 'thereunder','guideline', 'reference','onus','make','Page','Securities','Exchange','India'])

import os
import pandas as pd
from ast import literal_eval
from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline import QAPipeline

from cdqa.utils.download import download_model
download_model(model='bert-squad_1.1', dir='./models')

with open('glossary.json') as f:
    glossary = json.load(f)

/home/deepti-saravanan/.pyenv/versions/3.7.3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)



bert_qa.joblib already downloaded


In [5]:
def queryvocab(query):
    query1 = ""
    qvocab=[]
    for i in query.split():
        if(i not in stopwords.words()):
            if(i != 'What' and i != 'what' and i!='How'):
                qvocab.append(i)
                query1 = query1 + " " + i
    query = query1
    return qvocab, query

In [6]:
def regextract(docinput, docs, docregs, mainvocab, vocabdef, query, glossary):
    
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    v = []
    
    if(docinput == 'All'):
        rdocs = []
        for i in glossary.keys():
            if(i in query.lower()):
                dval = glossary[i]
                for j in dval:
                    if(j not in rdocs):
                        rdocs.append(j)
        if(len(rdocs)>0):
            docinput = []
            for i in rdocs:
                docinput.append(docs[i])
        else:
            docinput = ['All']
            
    else:
        d = [docinput]
        docinput = d
    
    for dval in docinput:
    
        if(dval != 'All'):
            ind = docs.index(dval)

            rules = docregs[ind]
        
            t=-1
            vocab0 = []
            for i in rules:
                t=t+1
                vocab0.append(i.split())
            
            for i in vocab0:
                for j in i:
                    if(len(j)>2):
                        for m in range(len(symbols)):
                            j = np.char.replace(j, symbols[m], '')
                            j = np.char.replace(j, ",", "")
                            j = np.array2string(j)
                            j = j.replace("'","")
                        if(j not in v):
                            v.append(j)
        else:
            rules = []
            for i in docregs:
                for j in i:
                    rules.append(j)
            vocab0 = mainvocab
        
            for j in vocab0:
                if(len(j)>2):
                    for m in range(len(symbols)):
                        j = np.char.replace(j, symbols[m], '')
                        j = np.char.replace(j, ",", "")
                        j = np.array2string(j)
                        j = j.replace("'","")
                    if(j not in v):
                        v.append(j)
    
    if('All' not in docinput):
        vdef = []
        for i in v:
            if(i in mainvocab and len(i)>2):
                vindex = mainvocab.index(i)
                vdef.append(vocabdef[vindex])
    else:
        vdef = vocabdef
        
    
    return rules, v, vdef, docinput

In [7]:
def querypreprocess(query, qvocab, definitions, finaltopics):
    if('regulations' in qvocab):
        qvocab.remove('regulations')
    if('rules' in qvocab):
        qvocab.remove('rules')
    if('rule' in qvocab):
        qvocab.remove('rule')
    if('chapter' in qvocab):
        qvocab.remove('chapter')
    if('section' in qvocab):
        qvocab.remove('section')
    if('sub' in qvocab):
        qvocab.remove('sub')
    if('SEBI' in qvocab):
        qvocab.remove('SEBI')
    if('means' in qvocab):
        qvocab.remove('means')
    if('shall' in qvocab):
        qvocab.remove('shall')
    if('Securities' in qvocab):
        qvocab.remove('Securities')
    if('Exchange' in qvocab):
        qvocab.remove('Exchange')
    if('pertaining' in qvocab):
        qvocab.remove('pertaining')
    if('India' in qvocab):
        qvocab.remove('India')
        
    query = ""
    for i in qvocab:
        query = query + " " + i
        
    synwords = []
    importantwords = []
    for i in qvocab:
        #print(i)
        flag = 0
        for j in definitions:
            if(i in j):
                flag = 1
                break
        if(flag == 1):
            importantwords.append(i)
        if(flag == 0):
            synwords.append(i)
    wforms = []
    for i in synwords:
        wforms.append(wordnet.morphy(i, wordnet.VERB))
        wforms.append(wordnet.morphy(i, wordnet.NOUN))
    for j in wforms:
        if(j != '' and j not in synwords and j != None):
            synwords.append(j)
    snew = []   
    for i in synwords:
        for j in finaltopics:
            for k in j:
                if(i != k):
                    snew.append(i)
    synwords = snew
    
    expansionwords = []
    for i in qvocab:
        flag=0
        for j in finaltopics:
            for k in j:
                if(i == k):
                    flag=1
        if(flag==1):
            if(i not in importantwords):
                importantwords.append(i)
        if(flag==0):
            for j0 in definitions:
                if(i in j0):
                    expansionwords.append(j0)
                    break
    
    qnew = query
    for j in expansionwords:
        k=0
        for i in definitions:
            if(i==j):
                s = i
                s = s + defvalues[k]
                qnew = qnew + ' ' + s
                break
            k=k+1
        else:
            qnew = qnew + ' ' + j
    query = qnew
    
    sent = nlp(query)
    t=0
    for token in sent:
        if(str(token) in importantwords):
            if(t!=0):
                if(str(sent[t-1]) not in importantwords):
                    importantwords.append(str(sent[t-1]))
            if(token.tag_ == 'VB'):
                importantwords.append(str(token))
        t+=1
    
    return query, qvocab, synwords, importantwords, expansionwords

## Regulations Retrieval Module

In [8]:
def legalembedding(synwords, modelvocab, m, vocab):
    
    synonyms0 = []
    
    for i in synwords:
        if(i in modelvocab):
            l = m.most_similar(i)
            for j in l:
                if(j in vocab):
                    synonyms0.append(j)
    return synonyms0

In [9]:
def defitionssynonyms(vocab, synwords, vdef, vocabdef, mainvocab):
    
    #Definitions of all synwords
    syndef = []
    for i in synwords:
        if(i == None):
            synwords.remove(i)
        else:
            syns = wordnet.synsets(i)
            if(syns != []):
                syndef.append(syns[0].definition())
    
    #Comparison of definitions
    synonyms1 = []
    if(syndef != []):
        p=0
        for i in syndef:
            for j in vdef:
                vind = vocabdef.index(j)
                if(synwords[p] in j):
                    if(mainvocab[vind] not in synonyms1):
                        synonyms1.append(mainvocab[vind])
            p=p+1
            
    return synonyms1

In [10]:
def similarityscore(synonyms0, synonyms1,synwords):
    synonyms2 = []
    for i in synonyms0:
        if(i not in synonyms2):
            synonyms2.append(i)
    for i in synonyms1:
        if(i not in synonyms2):
            synonyms2.append(i)
    
    synonyms = []
    for i in synwords:
        for j in synonyms2:
            score = []
            for syn in wordnet.synsets(i):
                for syn1 in wordnet.synsets(j):
                    score.append(syn.wup_similarity(syn1))
            if(None not in score):
                score.sort(reverse=True)
                if(len(score)>0):
                    if(score[0] > 0.5):
                        synonyms.append(j)
                
    return synonyms

In [11]:
def finalsynonyms(synwords, modelvocab, m, vocab, query, vdef, vocabdef, mainvocab):
    
    synonyms0 = legalembedding(synwords, modelvocab, m, vocab)
    synonyms1 = defitionssynonyms(vocab, synwords, vdef, vocabdef, mainvocab)
    synonyms = similarityscore(synonyms0, synonyms1,synwords)
    
    while(None in synonyms):
        synonyms.remove(None)
        
    for i in synonyms:
        for j in vocab:
            if(edit_distance(i,j) < 4):
                if(j not in synonyms):
                    synonyms.append(j)
                
    for i in synonyms:
        query = query + ' ' + i
        
    qvocab = []
    for i in query.split():
        qvocab.append(i)
    
    return synonyms, query, qvocab

In [12]:
def convert_lower_case(data):
    return np.char.lower(data)

def remove_stop_words(data):
    stop_words = STOPWORDS
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [13]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data)
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = stemming(data)
    return data

In [14]:
def tfidfreg(docregs, docinput, category):
    
    if(category == 'regulations'):
        d = [] 
    
        for i in docinput:
            if(i in docs):
                ind = docs.index(i)
                r0 = docregs[ind]
                d.append(r0)
            
        if(len(d)>0):
            docregs = d
        #print(docregs)

    regstring = []
    origregstring = []
    for i in docregs:
        for j in i:
            origregstring.append(j)
            data = preprocess(j)
            regstring.append(data)        
    wordregs = regstring
    
    t=0
    remind = []
    for i in regstring:
        if(i=='\xa0\n' or i == ''):
            remind.append(t)
            regstring.remove(i)
            wordregs.remove(wordregs[t])
        t+=1
        
    origreg = []
    t=0
    while(t<len(origregstring)):
        if(t not in remind):
            origreg.append(origregstring[t])
        t+=1
            
    wordregs = regstring
    
    DF = {}
    wlist = []
    for i in range(len(wordregs)):
        tokens = regstring[i]
        for w in tokens.split():
            try:
                DF[w].add(i)
            except:
                DF[w] = {i}
                wlist.append(w)
                
    for i in DF:
        DF[i] = len(DF[i])
    
    total_vocab = [x for x in DF]
    total_vocab_size = len(DF)
    
    tf_idf = {}
    N = len(regstring)
    for i in range(N):
        tokens = regstring[i].split()
        counter = Counter(tokens)
        for token in np.unique(tokens):
            if(token in wlist):
                tf = counter[token]/len(tokens)
                df = DF[token]
                idf = np.log((N+1)/(df+1))
                try:
                    tf_idf[token] = tf_idf[token] + tf*idf
                except:
                    tf_idf[token] = tf*idf                
            
    return DF, N, tf_idf, total_vocab, total_vocab_size, wordregs, regstring, origreg

In [ ]:
import pickle
with open('tfidf_regs.pkl','wb') as f:
    pickle.dump(tf_idf,f)

In [16]:
def q_vector(qvocab, N, total_vocab, DF):
    Q = np.zeros((len(total_vocab)))
    querytokens = qvocab
    counter = Counter(querytokens)
    words_count = len(querytokens)

    query_weights = {}
    
    for token in np.unique(querytokens):
        if(token in DF):
            tf = counter[token]/words_count
            df = DF[token]
            idf = math.log((N+1)/(df+1))

            try:
                ind = total_vocab.index(token)
                Q[ind] = tf*idf
            except:
                pass
    return Q

In [17]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [18]:
def cosine_similarity(k, query, total_vocab, N, qvocab, DF, origreg, D, importantwords):
        
    querytokensall = qvocab

    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    d_cosines = []
    
    query_vector = q_vector(qvocab, N, total_vocab, DF)
    
    t=0
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        t=t+1
        
    out = np.array(d_cosines).argsort()[::-1]
    
    answers = []
    for i in out:
        flag=0
        for j in querytokensall:
            if(j in origreg[i].split()):
                if(j in importantwords):
                    flag=flag+2
                else:
                    flag=flag+1
        if(flag>1):
            answers.append([flag,origreg[i]])
    answers.sort(reverse=True)

    
    if(k<len(answers)):
        return answers[0:k]
    else:
        return answers

In [20]:
def vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg,importantwords):
    
    D = np.zeros((N, len(total_vocab)))
    for i in tf_idf:
        try:
            ind = total_vocab.index(i[1])
            D[i[0]][ind] = tf_idf[i]
        except:
            pass
    
    A = cosine_similarity(k, query, total_vocab, N, qvocab, DF, origreg, D, importantwords)
    
    return A

In [21]:
def answermod(A, queryinput):
    
    ans = []
    ansreg = []
    
    t=0
    while(t<len(A)):
        A[t][0] = 'title'
        A[t][1] = [A[t][1]]
        t+=1
    t=0
    df = pd.DataFrame(A,columns=['title','paragraphs'])
    
    cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib')
    cdqa_pipeline.fit_retriever(df=df)
    
    prediction = cdqa_pipeline.predict(queryinput)
    
    return prediction[0], prediction[2]

In [24]:
def QnAmodel(queryinput, docinput,k,category):
    print('Preprocessing query......')
    qvocab, query = queryvocab(queryinput)
    if(category == 'regulations'):
        rules, vocab, vdef, docinput = regextract(docinput, docs, docregs, mainvocab, vocabdef, queryinput, glossary)
        query, qvocab, synwords, importantwords, expansionwords = querypreprocess(query, qvocab, definitions, finaltopics)
        print('tf-idf calculation in progress.....')
        DF, N, tf_idf, total_vocab, total_vocab_size, wordregs, regstring, origreg = tfidfreg(docregs,docinput,category)
        print('Extracting relevant answer regulations.......')
        A = vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg, importantwords)
        Aold = A
    elif(category == 'misc'):
        
        rules = ctext
        rules.extend(itext)
        
        #for lol in docregs:
        #    rules.extend(lol)
        vocab = miscvocab
        #vocab.extend(mainvocab)
        N = 447
        #N = 6379
        
        with open('misc_tf_idf.pkl', 'rb') as f:
            tf_idf = pickle.load(f)
        with open('misc_total_vocab.pkl', 'rb') as f:
            total_vocab = pickle.load(f)
        with open('misc_DF.json') as f:
            DF = json.load(f)
        with open('misc_origreg.pkl', 'rb') as f:
            origreg = pickle.load(f)
        
        query, qvocab, synwords, importantwords, expansionwords = querypreprocess(query, qvocab, definitions, finaltopics)
        print('Extracting relevant answer regulations.......')
        k = 5
        A = vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg, importantwords)
        Aold = A
        
        sentences = csent
        sentences.extend(isent)
        Anew = []
        for ans in A:
            ind = rules.index(ans[1])
            for sent in sentences[ind]:
                Anew.append([ans[0],sent])
        A = Anew
        
    
    #print('Finding synonyms and hypernyms......')
    #synonyms, query, qvocab = finalsynonyms(synwords, modelvocab, m, vocab, query, vdef, vocabdef, mainvocab)
    
    elif(category == 'legal case'):
        rules = ltext
        vocab = casevocab
        N = 1496
        
        with open('case_tf_idf.pkl', 'rb') as f:
            tf_idf = pickle.load(f)
        with open('case_total_vocab.pkl', 'rb') as f:
            total_vocab = pickle.load(f)
        with open('case_DF.json') as f:
            DF = json.load(f)
        with open('case_origreg.pkl', 'rb') as f:
            origreg = pickle.load(f)
        
        query, qvocab, synwords, importantwords, expansionwords = querypreprocess(query, qvocab, definitions, finaltopics)
        print('Extracting relevant answer regulations.......')
        k = 5
        A = vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg, importantwords)
        Aold = A
        
        sentences = lsent
        Anew = []
        for ans in A:
            ind = rules.index(ans[1])
            for sent in sentences[ind]:
                Anew.append([ans[0],sent])
        A = Anew
        
        
    else:
        lol = 0
        #Amendment queries
    
    print('Extracting answer')
    answer, ansreg = answermod(A, queryinput)
    return answer, ansreg, Aold

In [22]:
def QnAmodel(queryinput, docinput,k,category):
    print('Preprocessing query......')
    qvocab, query = queryvocab(queryinput)
    if(category == 'regulations'):
        rules, vocab, vdef, docinput = regextract(docinput, docs, docregs, mainvocab, vocabdef, queryinput, glossary)
        query, qvocab, synwords, importantwords, expansionwords = querypreprocess(query, qvocab, definitions, finaltopics)
        print('tf-idf calculation in progress.....')
        DF, N, tf_idf, total_vocab, total_vocab_size, wordregs, regstring, origreg = tfidfreg(docregs,docinput,category)
        print('Extracting relevant answer regulations.......')
        A = vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg, importantwords)
        Aold = A
    elif(category == 'misc'):
        
        rules = ctext
        rules.extend(itext)
        
        #for lol in docregs:
        #    rules.extend(lol)
        vocab = miscvocab
        #vocab.extend(mainvocab)
        N = 447
        #N = 6379
        
        with open('misc_tf_idf.pkl', 'rb') as f:
            tf_idf = pickle.load(f)
        with open('misc_total_vocab.pkl', 'rb') as f:
            total_vocab = pickle.load(f)
        with open('misc_DF.json') as f:
            DF = json.load(f)
        with open('misc_origreg.pkl', 'rb') as f:
            origreg = pickle.load(f)
        
        query, qvocab, synwords, importantwords, expansionwords = querypreprocess(query, qvocab, definitions, finaltopics)
        print('Extracting relevant answer regulations.......')
        k = 5
        A = vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg, importantwords)
        Aold = A
        
        sentences = csent
        sentences.extend(isent)
        Anew = []
        for ans in A:
            ind = rules.index(ans[1])
            for sent in sentences[ind]:
                Anew.append([ans[0],sent])
        A = Anew
        
    
    #print('Finding synonyms and hypernyms......')
    #synonyms, query, qvocab = finalsynonyms(synwords, modelvocab, m, vocab, query, vdef, vocabdef, mainvocab)
    
    elif(category == 'legal case'):
        rules = ltext
        vocab = casevocab
        N = 1496
        
        with open('case_tf_idf.pkl', 'rb') as f:
            tf_idf = pickle.load(f)
        with open('case_total_vocab.pkl', 'rb') as f:
            total_vocab = pickle.load(f)
        with open('case_DF.json') as f:
            DF = json.load(f)
        with open('case_origreg.pkl', 'rb') as f:
            origreg = pickle.load(f)
        
        query, qvocab, synwords, importantwords, expansionwords = querypreprocess(query, qvocab, definitions, finaltopics)
        print('Extracting relevant answer regulations.......')
        k = 5
        A = vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg, importantwords)
        Aold = A
        
        sentences = lsent
        Anew = []
        for ans in A:
            ind = rules.index(ans[1])
            for sent in sentences[ind]:
                Anew.append([ans[0],sent])
        A = Anew
        
        
    else:
        lol = 0
        #Amendment queries
    
    print('Extracting answer')
    #answer, ansreg = answermod(A, queryinput)
    return Aold

## User input

In [99]:
queryinput = "How does a company offer initial public offering"
docinput = 'All'
category = 'regulations'
A = QnAmodel(queryinput,docinput,10, category)

Preprocessing query......
tf-idf calculation in progress.....
Extracting relevant answer regulations.......
Extracting answer


/home/deepti-saravanan/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


In [100]:
Ans = []
for i in A:
    Ans.append(i[1])

In [101]:
test_queries.append(queryinput)
version1.append(Ans)

In [102]:
A = QnAmodel(queryinput,docinput,50, category)

Preprocessing query......
tf-idf calculation in progress.....
Extracting relevant answer regulations.......
Extracting answer


/home/deepti-saravanan/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


In [103]:
Ans = []
for i in A:
    Ans.append(i[1])

In [104]:
from rank_bm25 import BM25Okapi

corpus = Ans
tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

query = queryinput
tokenized_query = query.split(" ")

Ans1 = bm25.get_top_n(tokenized_query, corpus, n=10)

In [105]:
version2.append(Ans1)

In [107]:
answers_comp = []
t=0
while(t<len(test_queries)):
    answers_comp.append([test_queries[t], version1[t], version2[t]])
    t+=1

In [110]:
with open('answers_comparison.pkl','wb') as f:
    pickle.dump(answers_comp,f)

#### Concept paper + informal guidance

In [20]:
print(queryinput)
print('\n')
print(answer)
print('\n')
print(ansreg)

How to calculate the 5% creeping acquisition limit


only gross acquisitions shall be taken into account


c. For calculating the 5% creeping acquisition limit, as specified under regulation 3(2) of the Takeover Regulations, only gross acquisitions shall be taken into account. Any intermittent fall in shareholding owing to disposal of shares by the acquirer or dilution of shareholding on account of fresh issue of share capital shall be ignored.


In [28]:
print(queryinput)
print('\n')
print(answer)
print('\n')
print(ansreg)

What is the lock-in period duration in case of warrants


6 (Six) Month


3.1	The Lock-in period of 6 (Six) Month under Regulation 78(6) of ICDR Regulations, is to be determined on the "date of trading approval" under such provision. The warrants allotted to the warrant holder are not listed and hence concept of 'Trading Approval" shall not be applicable to such warrants, until the same are converted in to equity shares. Accordingly, the date relevant for calculation of lock-in period for pre preferential holding, under Regulation 78(6) of the ICDR


#### Regulations + CP + IG

In [44]:
print(queryinput)
print('\n')
print(answer)
print('\n')
print(ansreg)

How to calculate the 5% creeping acquisition limit


only gross acquisitions shall be taken into account


c. For calculating the 5% creeping acquisition limit, as specified under regulation 3(2) of the Takeover Regulations, only gross acquisitions shall be taken into account. Any intermittent fall in shareholding owing to disposal of shares by the acquirer or dilution of shareholding on account of fresh issue of share capital shall be ignored.


In [42]:
print(queryinput)
print('\n')
print(answer)
print('\n')
print(ansreg)

What is the lock-in period duration in case of warrants


6 (Six) Month


3.1	The Lock-in period of 6 (Six) Month under Regulation 78(6) of ICDR Regulations, is to be determined on the "date of trading approval" under such provision. The warrants allotted to the warrant holder are not listed and hence concept of 'Trading Approval" shall not be applicable to such warrants, until the same are converted in to equity shares. Accordingly, the date relevant for calculation of lock-in period for pre preferential holding, under Regulation 78(6) of the ICDR


In [72]:
print(queryinput)
print('\n')
print(answer)
print('\n')
print(ansreg)

How much amount will be provided as reward for informants in insider trading cases


10 lakh


Since the trading lot has been mandated as 10 lakh, participation in this platform is restricted to informed investors.


#### Only regulations

In [70]:
print(queryinput)
print('\n')
print(answer)
print('\n')
print(ansreg)

How much amount will be provided as reward for informants in insider trading cases


ten percent of the monetary sanctions collected or recovered and shall not exceed Rupees One crore


7D. (1) Upon collection or substantial recovery of the monetary sanctions amounting to at least twice the Reward, the Board may at its sole discretion, declare an Informant eligible for Reward and intimate the Informant or his or her legal representative to file an application in the format provided in Schedule-E for claiming such Reward: Provided that the amount of Reward shall be ten percent of the monetary sanctions collected or recovered  and  shall  not  exceed  Rupees  One  crore  or  such  higher  amount  as  the  Board  may specify from time to time: Provided  further  that  the  Board  may  if  deemed  fit,  out  of  the  total  Reward  payable,  grant  an interim reward not exceeding Rupees Ten lacs or such higher amount as the Board may specify from  time  to  time,  on  the  issue  of  final

#### Legal cases

In [44]:
print(queryinput)
print('\n')
print(answer)
print('\n')
print(ansreg)

What is the penalty amount for fraudulent and unfair trade activities


twenty-five crore rupees


6. In this regard the reliance on Hon’ble SAT order in the matter of Shailesh S. Jhaveri v. SEBI dated October, 4, 2012, wherein Hon’ble SAT examined the meaning of “disgorgement” is No applicable to the facts of this case, as the said order of Hon’ble SAT order is No in respect of liability of legal representatives for the disgorgement amount. 153. As observed earlier, Section 28B(2) of SEBI Act, 1992 enables SEBI to initiate action for disgorgement against the legal representatives after the death of the deceased. As per the aforesaid legal provisions, in conjunction with general law, and case laws cited above, I am Order in the matter of Kirloskar Brothers Limited Page 103 of 113 of the view that proceeding for disgorgement for unlawful gains loss avoided by Late Noticee No 6 survives his death. Therefore, proceeding for disgorgement can be initiated against LRs of Noticee No 6 and the

In [46]:
print(queryinput)
print('\n')
print(answer)
print('\n')
print(ansreg)

What is the penalty amount if Noticee had acquired or disposed shares for which he was required to make disclosures


ten lakh rupees


7. (2) Continual Disclosures. (a). Every promoter, employee and director of every company shall disclose to the company the number of such securities acquired or disposed of within two trading days of such transaction if the value of the securities traded, whether in one transaction or a series of transactions over any calendar quarter, aggregates to a traded value in excess of ten lakh rupees or such other value as may be specified; Issue I: Whether Noticee has violated Regulation 7(2)(a) of PIT Regulations? Findings:


In [61]:
print(queryinput)
print('\n')
print(answer)
print('\n')
print(ansreg)

What is the penalty amount if Noticee had acquired or disposed shares for which he was required to make disclosures under the provisions of regulations 7(1) & (2) of the SAST Regulations


0.17%


81. It was also alleged that duruing the same period, the Noticee No 2 had acquired 5.34% shares of the Velan Hotels and his holdings crossed 5% on November 26, 2009. As per regulation 13(1) of PIT Regulations and regulation 7(1) of SAST Regulations, the Noticee No 2 was required to make disclosure to the Company Velan Hotels and to the Stock Exchange where shares of the target company are listed, which he allegedly failed to make such disclosures. Further, during the period December 2009 to March 2010, the shareholding of Noticee No 2 had come down from 5.34% to 0.17% (i.e. change was more than 2% on March 17, 2010) and as per regulation 13 (3) of PIT Regulations, Noticee No 2 was required to make disclosures to the Company Velan Hotels which he had allegedly failed to disclose the same.


#### KG info for legal team validation

In [113]:
with open('/home/deepti-saravanan/Desktop/kg_connections/conceptpaper_graph.pkl','rb') as f:
    cp = pickle.load(f)

In [115]:
with open('/home/deepti-saravanan/Desktop/kg_connections/ig.pickle','rb') as f:
    ig = pickle.load(f)

In [117]:
with open('/home/deepti-saravanan/Desktop/kg_connections/lc.pickle','rb') as f:
    lc = pickle.load(f)

In [131]:
import pandas as pd
lc_df = pd.DataFrame(lc, columns = ['Regulatory Document','Regulation number','file_name','Topic represented by edges in graph'])

In [133]:
lc_df.to_csv('legal_cases_graphinfo.csv',index=False)